In [27]:
import numpy as np
import scipy
from scipy.integrate import quad
from scipy.stats import multivariate_normal, norm
from scipy.optimize import minimize

# Equivalent to R's logit function
def logit(x):
    return np.log(x / (1 - x))

# Equivalent to R's expit function
def expit(x):
    return 1 / (1 + np.exp(-x))
# Bivariate normal PDF
# Equivalent to R's biv_pdf function
def biv_pdf(x, y, mu, sigma, rho, log=False):
    rho_part = 1 - rho**2
    if rho_part <= 0:
        rho_part = 0.000001  # To prevent division by zero or negative values

    factor_part = 2 * np.pi * sigma[0] * sigma[1] * np.sqrt(rho_part)
    exp_part = ((x - mu[0]) / sigma[0])**2 + ((y - mu[1]) / sigma[1])**2
    exp_part -= 2 * rho * (x - mu[0]) * (y - mu[1]) / (sigma[0] * sigma[1])
    exp_part = -exp_part / (2 * rho_part)

    if log:
        return exp_part - np.log(factor_part)
    else:
        return np.exp(exp_part) / factor_part

# Equivalent to R's dmg_int_inner function
def dmg_int_inner(x, y, mu, sigma, rho, eta, alpha, l):
    sd_x = sigma[0]
    sd_y = sigma[1]
    # Adjusted value of y
    adjusted_y = y + (mu[1] / mu[0]) * alpha * (l / eta - x)
    return biv_pdf(x, adjusted_y, mu, sigma, rho)

# Equivalent to R's dmg_int function
def dmg_int(y, mu, sigma, rho, eta, alpha, l):
    # Use scipy's quad to perform the integration
    result, _ = quad(dmg_int_inner, l, l / eta, args=(y, mu, sigma, rho, eta, alpha, l), epsabs=1e-5)
    return result


def dmg_model(samples, eta, alpha, l, mu):
    # Check if l > samples[:, 0] * eta for each row
    condition = l > samples[:, 0] * eta
    adjusted_values = samples[:, 1] - (mu[1] / mu[0]) * alpha * (l / eta - samples[:, 0])
    
    # If condition is True, calculate damage; otherwise, use undamaged value
    return np.where(condition, adjusted_values, samples[:, 1])
def int_function(y_star, mu, sigma, rho, l):
    a_l = (l - mu[0] - rho * (sigma[0] / sigma[1]) * (y_star - mu[1])) / (sigma[0] * np.sqrt(1 - rho**2))
    return norm.pdf(y_star, loc=mu[1], scale=sigma[1]) * (1 - norm.cdf(a_l))


def PFY_lik(mu, sigma, rho, eta, alpha, l, data):
    part_1 = np.sum(norm.logpdf(data[data[:, 2] == 1, 0], loc=mu[0], scale=sigma[0]))
    
    dmg_int_values = np.array([dmg_int(y, mu, sigma, rho, eta, alpha, l) for y in data[data[:, 2] == 0, 1]])
    int_function_values = np.array([int_function(y, mu, sigma, rho, 1/eta * l) for y in data[data[:, 2] == 0, 1]])
    
    part_2 = np.sum(np.log(dmg_int_values + int_function_values))
    
    return part_1 + part_2


def pl_gen(mu, sigma, rho, eta, alpha, l, N):
    # Covariance matrix
    cov_matrix = np.array([
        [sigma[0]**2, sigma[0] * sigma[1] * rho],
        [sigma[0] * sigma[1] * rho, sigma[1]**2]
    ])
    # Generate samples from bivariate normal distribution
    samples = np.random.multivariate_normal(mean=mu, cov=cov_matrix, size=N)
    res = np.zeros((samples.shape[0], samples.shape[1] + 1))
    
    # Fail in the proof loading
    fail_ids = samples[:, 0] < l
    res[fail_ids, 0] = samples[fail_ids, 0]
    res[fail_ids, 2] = 1
    
    # Survive in the proof loading
    survive_ids = samples[:, 0] >= l
    res[survive_ids, 1] = dmg_model(samples[survive_ids], eta, alpha, l, mu)
    res[survive_ids, 2] = 0
    
    return res


In [28]:
# Example usage (requires biv_pdf to be implemented)
mu = [45, 5.5]
sigma = [13, 1]
rho = .7
eta = .7
alpha = 0

# rho = correlation_coefficient
N = 87

R_pf = norm.ppf([0.2, 0.4, 0.6], loc=mu[0], scale=sigma[0])
T_pf = norm.ppf([0.2, 0.4, 0.6], loc=mu[1], scale=sigma[1])
l = R_pf[2]
pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
#print(pf_data)

In [29]:
theta0  = [mu[0],mu[1],
           sigma[0],sigma[1],logit(rho),logit(eta),alpha]


In [30]:
PFY_lik(mu, sigma, rho, eta, alpha, l, pf_data)

-274.7532997083696

In [31]:
def alpha_checkR(theta, R_group, T_group, R_pl, T_pl, shoulder_group):
    mu = theta[0:2]
    sigma = theta[2:4]
    rho = expit(theta[4])
    eta = expit(theta[5])
    alpha = theta[6]
    
    # Likelihood calculations
    lik = (PFY_lik(mu, sigma, rho, 1, 0, R_pl, R_group) +
           PFY_lik([mu[1], mu[0]], [sigma[1], sigma[0]], rho, eta, alpha, T_pl, T_group) +
           np.sum(norm.logpdf(shoulder_group[0], loc=mu[0], scale=sigma[0])) +
           np.sum(norm.logpdf(shoulder_group[1], loc=mu[1], scale=sigma[1])))
    
    # Handle infinite likelihood
    if np.isinf(lik):
        lik = -10000
    
    return -1 * lik

def single_fitalpha(theta, group, group_pl, group_name, shoulder_group):
    mu = theta[0:2]
    sigma = theta[2:4]
    rho = theta[4]
    eta = theta[5]
    alpha = theta[6]
    
    if group_name == "R":
        lik = PFY_lik(mu, sigma, rho, eta, alpha, group_pl, group)
    elif group_name == "T":
        lik = PFY_lik([mu[1], mu[0]], [sigma[1], sigma[0]], rho, eta, alpha, group_pl, group)
    
    lik += (np.sum(norm.logpdf(shoulder_group[0], loc=mu[0], scale=sigma[0])) +
            np.sum(norm.logpdf(shoulder_group[1], loc=mu[1], scale=sigma[1])))
    
    if np.isinf(lik):
        lik = -10000
    
    return -1 * lik

# Equivalent to R's single_alpha0 function
def single_alpha0(theta, group, group_pl, group_name, shoulder_group):
    mu = theta[0:2]
    sigma = theta[2:4]
    rho = expit(theta[4])
    
    if group_name == "R":
        lik = PFY_lik(mu, sigma, rho, 1, 0, group_pl, group)
    elif group_name == "T":
        lik = PFY_lik([mu[1], mu[0]], [sigma[1], sigma[0]], rho, 1, 0, group_pl, group)
    
    lik += (np.sum(norm.logpdf(shoulder_group[0], loc=mu[0], scale=sigma[0])) +
            np.sum(norm.logpdf(shoulder_group[1], loc=mu[1], scale=sigma[1])))
    
    if np.isinf(lik):
        lik = -10000
    
    return -1 * lik



def ecdf(x):
    x = np.sort(x)
    n = len(x)
    def _ecdf(v):
        # side='right' because we want Pr(x <= v)
        return (np.searchsorted(x, v, side='right') + 1) / n
    return _ecdf

In [32]:
np.random.seed(42)  # Set seed for reproducibility
pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
shoulder_group = [R100_data, T100_data]


In [39]:
def lrt_fit(jj,theta):
    np.random.seed(jj)  # Set seed for reproducibility
    pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
    R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
    T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
    shoulder_group = [R100_data, T100_data]



    optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group),
                           method = "Nelder-Mead")
    llr_checkR = 2 * optim_checkR.fun
    theta_est = optim_checkR.x
    # # Optimization step for single_fitalpha
    bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.01, 0.99), (0.01, 0.99), (-2, 10)]
    optimout = minimize(single_fitalpha, theta0, 
                        args=(pf_data,l, "R", shoulder_group), 
                        method="L-BFGS-B", bounds=bounds)
    llr_full = 2 * optimout.fun
    llr_stat = llr_checkR - llr_full
    return [jj,llr_stat,*theta_est.tolist()]


In [64]:
def lrt_sim(jj,theta):
    np.random.seed(jj)  # Set seed for reproducibility
    mu = theta[0:2]
    sigma = theta[2:4]
    rho = expit(theta[4])
    pf_data = pl_gen(mu, sigma, rho, 1,0, l, N)
    R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
    T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
    shoulder_group = [R100_data, T100_data]



    optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group),
                           method = "Nelder-Mead")
    llr_checkR = 2 * optim_checkR.fun
    theta_est = optim_checkR.x
    # # Optimization step for single_fitalpha
    bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.03, 0.97), (0.03, 0.97), (-2, 10)]
    optimout = minimize(single_fitalpha, theta0, 
                        args=(pf_data,l, "R", shoulder_group), 
                        method="L-BFGS-B", bounds=bounds)
    llr_full = 2 * optimout.fun
    llr_stat = llr_checkR - llr_full
    return [jj,llr_stat,*theta_est.tolist()]


In [35]:
lrt_fit(1,theta0)

[1,
 0.07975053339305305,
 44.92814667465602,
 5.5244981584826585,
 12.957847692048464,
 0.9989045444712349,
 0.7168621583922781]

In [36]:
expit(0.7168)

0.6719019664234913

In [37]:
import joblib
N_rep = 200
numbers = range(N_rep)
results = joblib.Parallel(n_jobs=8)(joblib.delayed(lrt_fit)(num,theta0) for num in numbers)

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.p

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log


In [38]:
res = np.array(results)
res[:,1]

array([ 2.08070794e+00,  7.97505334e-02,  2.90162115e+00,  3.86050712e+00,
        1.10952673e+01,  1.18012387e-01,  4.32647497e-02,  2.36368214e-06,
        2.75785327e+00, -2.77082439e-04, -3.03375855e-05, -2.81982843e-05,
        1.47492610e+00, -4.51201943e-01,  2.24524437e-01,  5.59007396e-01,
        5.61072974e+00,  2.09319510e-01,  3.72705768e-01,  1.62632620e+00,
        1.06240781e+00,  5.74387986e-01,  5.15494081e-04,  3.77173106e+00,
        8.77157106e-01,  4.38647082e-01,  8.66060188e+00,  3.37346909e-01,
       -7.06301898e-06,  3.55133593e-01,  8.63521815e-01,  2.96239670e+00,
        4.24729728e-01,  6.47651421e+00,  2.70268272e+00,  4.21892314e+00,
        3.65334504e+00,  1.39910799e-06,  2.24367609e+00,  5.74599079e-01,
        7.35570761e+00, -1.69040138e-04,  1.16039971e-01,  1.79477355e-01,
       -1.34830058e-04,  5.11774292e-01,  9.68093454e-01, -2.88046645e-05,
       -6.34145908e-05,  7.45685899e-01, -9.67862143e-05,  2.66676261e+00,
        1.94226528e+00,  

In [87]:
p_val_array = np.array([ 2.08070794e+00,  7.97505334e-02,  2.90162115e+00,  3.86050712e+00,
        1.10952673e+01,  1.18012387e-01,  4.32647497e-02,  2.36368214e-06,
        2.75785327e+00, -2.77082439e-04, -3.03375855e-05, -2.81982843e-05,
        1.47492610e+00, -4.51201943e-01,  2.24524437e-01,  5.59007396e-01,
        5.61072974e+00,  2.09319510e-01,  3.72705768e-01,  1.62632620e+00,
        1.06240781e+00,  5.74387986e-01,  5.15494081e-04,  3.77173106e+00,
        8.77157106e-01,  4.38647082e-01,  8.66060188e+00,  3.37346909e-01,
       -7.06301898e-06,  3.55133593e-01,  8.63521815e-01,  2.96239670e+00,
        4.24729728e-01,  6.47651421e+00,  2.70268272e+00,  4.21892314e+00,
        3.65334504e+00,  1.39910799e-06,  2.24367609e+00,  5.74599079e-01,
        7.35570761e+00, -1.69040138e-04,  1.16039971e-01,  1.79477355e-01,
       -1.34830058e-04,  5.11774292e-01,  9.68093454e-01, -2.88046645e-05,
       -6.34145908e-05,  7.45685899e-01, -9.67862143e-05,  2.66676261e+00,
        1.94226528e+00,  1.58805791e+00,  4.01604989e+00,  2.82920423e-01,
        1.14566627e-01,  5.25558175e-01,  3.05603953e-06,  2.95037748e-04,
        1.35461768e+00,  7.23723770e-01,  1.26344268e-02,  8.00758550e+00,
        5.95498297e-01,  3.60823807e-01,  2.22944845e+00, -3.77751635e-05,
        1.26993327e+00,  3.52381216e+00,  7.45651583e-01,  5.92805418e-06,
        4.33498844e-01,  8.85199125e-01,  1.98220391e+00,  1.05510480e+00,
        1.42293571e+00,  5.18007467e+00,  2.21901096e+00,  6.65186168e-01,
        4.21744174e+00,  8.20816123e+00,  4.44961917e+00,  1.76697354e+00,
        3.03204616e+00,  4.51366504e-01,  1.42456096e+00,  2.90702220e+00,
        2.97237227e+00,  4.99811214e-01,  1.50392373e-01,  4.98448366e-01,
        1.13972701e+00,  8.84799735e-01,  1.18878932e-01,  3.60720946e+00,
        4.05791167e+00,  2.61778024e+00,  1.19256199e+00,  1.18400739e+00,
        5.03643810e+00, -2.13661406e-06, -7.50981735e-06,  4.16867021e+00,
        8.09619745e-01, -3.96279306e-05,  5.03627293e+00,  5.17486474e+00,
        1.28340077e-01,  4.06999462e+00,  7.40650973e-01, -5.93633104e-06,
        1.11213851e+00,  3.45311160e+00, -2.02474489e-05,  1.27200701e+00,
        7.46307858e+00, -2.19031822e-05,  1.68851209e-01,  7.44309210e-01,
        9.96078983e-01,  1.50334930e+00,  3.39589380e+00,  1.09465841e-02,
        2.55801119e+00,  7.81828424e-01,  9.28333809e-01,  3.56795845e+00,
        1.66186994e+00,  6.22710104e-07,  4.00396500e-01,  8.10480823e-01,
       -2.35922530e-05,  1.35845153e-01,  2.39896017e-01,  1.57608653e-02,
        9.24960210e-01,  6.27022327e-01,  1.50289476e+00, -1.43644893e-05,
        1.45852749e+00,  2.30794142e-01,  1.76095657e+00,  3.93037716e-01,
        1.98919262e+00,  1.20268603e+00,  6.61213880e-06,  2.17850453e-01,
        3.66985174e-01,  3.12136561e+00,  3.45813349e-01,  4.00720612e+00,
        6.80584041e-01,  1.89141665e-01,  3.65599128e+00,  2.60473539e-01,
        3.79127116e-01,  1.40749781e-01,  4.42043165e-06,  3.60515828e-01,
        2.19647695e+00,  6.11855059e-03,  1.54846859e+00,  3.92766580e-01,
        7.06711430e-01,  9.73743289e-01,  4.14577733e-01,  2.36871381e+00,
        3.01105996e-01,  5.17745101e-01,  4.32556033e+00, -1.48565309e-05,
        1.14759113e+00,  4.09672349e+00,  9.98035767e-01,  8.64862008e-01,
        8.51281209e-02,  2.82008491e+00,  1.56386991e+00,  2.22902730e+00,
        1.60006569e+00,  5.12908215e-02,  3.39581614e+00,  1.03556998e+00,
       -1.24045391e-06,  6.06855371e+00,  9.66199132e+00,  1.89736209e+00,
        4.40765551e-01,  7.97604970e-02,  6.96928869e-01,  1.08283915e+00,
       -2.34176332e-06,  3.77556464e-01, -8.79769505e-06,  3.20354362e+00,
        7.01525230e-01,  9.75862534e-01,  3.75394622e+00,  3.18397366e+00])

np.mean(p_val_array < 4.5)

0.93

In [147]:
np.where((p_val_array < 5.2) & (p_val_array > 5))


(array([ 77, 100, 106, 107]),)

In [106]:
np.where((p_val_array < 4.8) & (p_val_array > 4))


(array([ 35,  54,  80,  82,  96, 103, 109, 151, 170, 173]),)

In [151]:
p_val_array[107]
#35 96 

5.17486474

In [141]:
ii = 35
np.random.seed(ii)  # Set seed for reproducibility
pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
shoulder_group = [R100_data, T100_data]



optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group),
                       method = "Nelder-Mead")
llr_checkR = 2 * optim_checkR.fun
theta_est = optim_checkR.x
# # Optimization step for single_fitalpha
bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.01, 0.99), (0.01, 0.99), (-2, 10)]
optimout = minimize(single_fitalpha, theta0, 
                    args=(pf_data,l, "R", shoulder_group), 
                    method="L-BFGS-B", bounds=bounds)
llr_full = 2 * optimout.fun
llr_stat = llr_checkR - llr_full

N_sim = 500
numbers = range(N_sim,1000)
results = joblib.Parallel(n_jobs=8)(joblib.delayed(lrt_sim)(num,theta_est) for num in numbers)
p_val = 1-ecdf(np.array(results)[:,1])(llr_stat )
print(p_val)

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.p

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.p

0.05800000000000005


In [142]:
print(p_val)

0.05800000000000005


In [161]:
1-ecdf(np.array(results)[:,1])(-0.002 )

0.978

In [42]:
res = np.zeros((200,3))
for ii in range(1):
    np.random.seed(ii)  # Set seed for reproducibility
    pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
    R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
    T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
    shoulder_group = [R100_data, T100_data]



    optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group),
                           method = "Nelder-Mead")
    llr_checkR = 2 * optim_checkR.fun
    theta_est = optim_checkR.x
    # # Optimization step for single_fitalpha
    bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.01, 0.99), (0.01, 0.99), (-2, 10)]
    optimout = minimize(single_fitalpha, theta0, 
                        args=(pf_data,l, "R", shoulder_group), 
                        method="L-BFGS-B", bounds=bounds)
    llr_full = 2 * optimout.fun
    llr_stat = llr_checkR - llr_full
    
    N_sim = 200
    numbers = range(N_sim)
    results = joblib.Parallel(n_jobs=8)(joblib.delayed(lrt_sim)(num,theta_est) for num in numbers)
    p_val = 1-ecdf(np.array(results)[:,1])(llr_stat )
    res[ii,:] = [ii,llr_stat,p_val]


/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value en

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/fol

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/fol

...

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/fol

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/fol

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:42: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: divide by zero encountered in scalar divide
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7795/1550408679.py:49: RuntimeWarning: invalid value encountered in multiply
/var/fol

In [65]:
def lrt_sim(jj,theta):
    np.random.seed(jj)  # Set seed for reproducibility
    mu = theta[0:2]
    sigma = theta[2:4]
    rho = expit(theta[4])
    pf_data = pl_gen(mu, sigma, rho, 1,0, l, N)
    R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
    T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
    shoulder_group = [R100_data, T100_data]



    optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group),
                           method = "Nelder-Mead")
    llr_checkR = 2 * optim_checkR.fun
    theta_est = optim_checkR.x
    # # Optimization step for single_fitalpha
    bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.03, 0.97), (0.03, 0.97), (-2, 10)]
    optimout = minimize(single_fitalpha, theta0, 
                        args=(pf_data,l, "R", shoulder_group), 
                        method="L-BFGS-B", bounds=bounds)
    llr_full = 2 * optimout.fun
    llr_stat = llr_checkR - llr_full
    return [jj,llr_stat,*theta_est.tolist()]

lrt_sim(0,theta_est)

[0,
 3.796499420165219,
 42.57320622470884,
 5.401082203786954,
 12.894277944762223,
 1.0535351712774887,
 0.9948648335978141]